In [1]:
import emcee
from scipy.optimize import minimize
import corner
import ptemcee
import subprocess
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from emukit.core import ParameterSpace, ContinuousParameter
from GPy.models import GPRegression
from emukit.model_wrappers import GPyMultiOutputWrapper
from GPy.kern import RBF

1-dimensional input, 2-dimensional outputs

In [2]:
parameter_space = ParameterSpace([ContinuousParameter('x', 0, 1)])

In [3]:
from emukit.core.initial_designs import RandomDesign

design = RandomDesign(parameter_space)
num_data_points = 2
X = design.get_samples(num_data_points)

In [4]:
def f(X): # add noise
    y1 = np.sin(X)
    y2 = np.cos(X)
    return np.hstack([np.sin(X),np.cos(X)])

In [5]:
Y = f(X)
kernel = RBF(input_dim=1)
model_gpy = GPRegression(X,Y,kernel=kernel, noise_var=0.01**2)
model_emukit = GPyMultiOutputWrapper(model_gpy, n_outputs=2, n_optimization_restarts=1)

In [6]:
model_emukit.predict(np.array(0.5).reshape(1,-1))

(array([[0.47849071, 0.87194918]]), array([[0.00463798]]))

In [7]:
model_emukit.predict_covariance(np.array(0.5).reshape(1,-1)) # why just 1 variance?

array([[0.00463798]])

In [8]:
from emukit.experimental_design.acquisitions.contour_MultiObs import contourWeightedEI

In [9]:
a_list = [0.2, 0.5]
alpha_list = [2, 2]
w_list = [0.5, 0.5]

In [10]:
contour_MD = contourWeightedEI(models = model_emukit, a_list = a_list, alpha_list = alpha_list, w_list = w_list)